<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/autocovariances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

ql.Array(10, 1.0, 1.0): Creates the input array [1.0, 2.0, ..., 10.0].
ql.Array(list_of_values): Used to create ql.Array from Python lists for expected values.
ql.convolutions(x, conv_out, max_lag): The Python function takes the input array, an output array (which it fills), and the max lag.
mean = ql.autocovariances(x, acovf_out, max_lag, center_flag): Similarly, acovf_out is filled, and the function returns the mean of x.
mean = ql.autocorrelations(x_in_out, acorf_out, max_lag, center_flag): If center_flag is True, the input array x_in_out is modified in-place to be centered around its mean. This is important for the last check in testAutoCorrelations.
ql.DotProduct(array1, array2): Used for the sum of squared differences check.
Error Reporting: Uses self.fail() with f-strings for detailed messages if DotProduct check fails, and self.assertAlmostEqual or a custom self.assertArrayAlmostEqual for individual element/value checks.
testAutoCorrelations Variance Note: The C++ test's first expected value for autocorrelations (when center=true) is the unbiased sample variance. The Python ql.autocorrelations function mirrors this behavior: it calculates the biased variance from the centered data and then, if N > 1, multiplies it by N/(N-1) to store the unbiased sample variance as the first element of the output array. The subsequent elements are the autocorrelation coefficients. The Python code comments and implementation reflect this understanding.


In [ ]:
import QuantLib as ql
import unittest
import math # For math.fabs, though abs() works too

class AutocovariancesTests(unittest.TestCase):

    def assertArrayAlmostEqual(self, arr1, arr2, delta, msg=""):
        self.assertEqual(len(arr1), len(arr2), f"{msg} Arrays have different lengths.")
        for i in range(len(arr1)):
            self.assertAlmostEqual(arr1[i], arr2[i], delta=delta,
                                   msg=f"{msg} Element {i} differs: {arr1[i]} vs {arr2[i]}")

    def test_convolutions(self):
        print("Testing convolutions...")
        x = ql.Array(10, 1.0, 1.0) # [1.0, 2.0, ..., 10.0]
        conv_calculated = ql.Array(6) # Output array

        # ql.convolutions(const Array& x, Array& conv, Size max_lag)
        ql.convolutions(x, conv_calculated, 5) # max_lag is 5 (0-based, so lags 0 to 4)
                                              # conv_calculated will have size max_lag + 1 = 6

        expected_values = [385.0, 330.0, 276.0, 224.0, 175.0, 130.0]
        expected_array = ql.Array(expected_values)

        # Check using DotProduct of differences
        delta = conv_calculated - expected_array
        dot_product_delta = ql.DotProduct(delta, delta)

        if dot_product_delta > 1.0e-6:
            self.fail(
                f"Convolution error:\n"
                f"  Calculated: {list(conv_calculated)}\n"
                f"  Expected:   {list(expected_array)}\n"
                f"  DotProduct(delta, delta): {dot_product_delta:.4e}"
            )
        # Also do element-wise assertion for clearer individual failures if any
        self.assertArrayAlmostEqual(conv_calculated, expected_array, delta=1e-7, msg="Convolution values differ.")

    def test_auto_covariances(self):
        print("Testing auto-covariances...")
        x = ql.Array(10, 1.0, 1.0) # [1.0, 2.0, ..., 10.0]
        acovf_calculated = ql.Array(6) # Output array for autocovariances

        # Real autocovariances(const Array& x, Array& acovf, Size max_lag, bool center = true)
        # Here, center is false
        mean_calculated = ql.autocovariances(x, acovf_calculated, 5, False)

        expected_mean = 5.5
        expected_acovf_values = [8.25, 6.416666666666667, 4.25, 1.75, -1.0833333333333333, -4.25]
        expected_acovf_array = ql.Array(expected_acovf_values)

        if abs(mean_calculated - expected_mean) > 1.0e-6:
            self.fail(
                f"Mean error (autocovariances):\n"
                f"  Calculated: {mean_calculated}\n"
                f"  Expected:   {expected_mean}"
            )
        self.assertAlmostEqual(mean_calculated, expected_mean, delta=1.0e-6, msg="Calculated mean from autocovariances differs.")

        delta_acovf = acovf_calculated - expected_acovf_array
        dot_product_delta_acovf = ql.DotProduct(delta_acovf, delta_acovf)

        if dot_product_delta_acovf > 1.0e-6:
            self.fail(
                f"Autocovariances error:\n"
                f"  Calculated acovf: {list(acovf_calculated)}\n"
                f"  Expected acovf:   {list(expected_acovf_array)}\n"
                f"  DotProduct(delta, delta): {dot_product_delta_acovf:.4e}"
            )
        self.assertArrayAlmostEqual(acovf_calculated, expected_acovf_array, delta=1e-6, msg="Autocovariance values differ.")


    def test_auto_correlations(self):
        print("Testing auto-correlations...")
        x_original = ql.Array(10, 1.0, 1.0) # [1.0, 2.0, ..., 10.0]
        # The autocorrelations function with center=true will modify x in place.
        # So, we pass a copy if we need to check x_original later or use it unmodified.
        # In this C++ test, 'x' is modified.
        x_for_autocorr = ql.Array(x_original)

        acorf_calculated = ql.Array(6) # Output array for autocorrelations

        # Real autocorrelations(Array& x, Array& acorf, Size max_lag, bool center = true)
        # Here, center is true, so x_for_autocorr will be centered.
        mean_calculated = ql.autocorrelations(x_for_autocorr, acorf_calculated, 5, True)

        expected_mean = 5.5
        # Note: First element of expected is variance if center=true and data is centered.
        # QL C++ returns sum_sq_dev / N. Python ql.autocorrelations first element is (sum_sq_dev / N) * N / (N-1) if biased=False by default, which is not the case here.
        # The C++ test's 'expected' array's first element implies the sample variance (biased, divided by N).
        # Let's re-check QL behavior or documentation for the first element of acorf.
        # From QL source (autocovariances.cpp for autocorrelations with center=true):
        # acorf[0] = sum_sq_dev / N; -- this is variance (biased)
        # then acorf[k] = acovf[k] / acorf[0] for k > 0
        # So, the C++ test's expected values for autocorrelation are correct under this interpretation.
        # The value 9.166667 is ( (10^2-1)/12 ) * (10/9) (variance of 1..10, then scaled for sample?)
        # Let's use the C++ test's expected values as ground truth for the QL function's output.
        # Variance of 1..10 is (10^2-1)/12 = 99/12 = 8.25.
        # The first expected value is 9.166667. This seems to be the (biased) sample variance * (N / (N-1)) if it were unbiased.
        # Let's trace the C++ source if `acorf[0]` is variance or something else when center=true.
        # `autocorrelations` when `center == true`:
        # 1. Centers `x` around `mean`.
        # 2. Calls `autocovariances(x, acorf, max_lag, false)` (since x is already centered).
        #    - `acovf[0]` becomes sum( (x_i - mean)^2 ) / N  (this is variance, biased)
        # 3. Then, for k > 0, `acorf[k] /= acorf[0]`.
        # So, `acorf[0]` should be the biased variance. For x=[1..10], mean=5.5.
        # Deviations: -4.5, -3.5, ..., 3.5, 4.5
        # Sum of squared deviations: 2 * (4.5^2 + 3.5^2 + 2.5^2 + 1.5^2 + 0.5^2) = 2 * (20.25 + 12.25 + 6.25 + 2.25 + 0.25) = 2 * 41.25 = 82.5
        # Biased variance: 82.5 / 10 = 8.25.
        # So, the first element of expected (9.166667) in C++ test is confusing if it's meant to be acorf[0].
        # If the C++ test `expected` is indeed the output `acorf`, then the QL C++ `autocorrelations`
        # must be putting something other than simple variance in `acorf[0]`.

        # Re-reading C++ utilities.hpp `autocorrelations` when `center == true`:
        # It first calculates `mean` and `acovf[0]` (the variance `var`).
        # Then it normalizes `acovf[k]` by `var` for `k > 0`.
        # It *then* sets `acovf[0] = var * N / (N-1)` IF N > 1. This makes it the *unbiased* sample variance.
        # N = 10, so N/(N-1) = 10/9.
        # Unbiased sample variance = 8.25 * (10/9) = 82.5 / 9 = 9.1666666...
        # This matches the C++ test's first expected value. So, `acorf_calculated[0]` will be the unbiased sample variance.

        expected_acorf_values = [
            9.166666666666666, # Unbiased sample variance
            0.7777777777777778,
            0.5151515151515151,
            0.2121212121212121,
           -0.13131313131313133,
           -0.5151515151515151
        ]
        expected_acorf_array = ql.Array(expected_acorf_values)

        if abs(mean_calculated - expected_mean) > 1.0e-6:
            self.fail(
                f"Mean error (autocorrelations):\n"
                f"  Calculated: {mean_calculated}\n"
                f"  Expected:   {expected_mean}"
            )
        self.assertAlmostEqual(mean_calculated, expected_mean, delta=1.0e-6, msg="Calculated mean from autocorrelations differs.")

        delta_acorf = acorf_calculated - expected_acorf_array
        dot_product_delta_acorf = ql.DotProduct(delta_acorf, delta_acorf)

        if dot_product_delta_acorf > 1.0e-6:
            self.fail(
                f"Autocorrelations error:\n"
                f"  Calculated acorf: {list(acorf_calculated)}\n"
                f"  Expected acorf:   {list(expected_acorf_array)}\n"
                f"  DotProduct(delta, delta): {dot_product_delta_acorf:.4e}"
            )
        self.assertArrayAlmostEqual(acorf_calculated, expected_acorf_array, delta=1e-7, msg="Autocorrelation values differ.") # Slightly higher delta for first element due to division.

        # Test the centering of x_for_autocorr (which was modified in place)
        # Original x was [1, 2, ..., 10]. Mean is 5.5.
        # Centered x should be [1-5.5, 2-5.5, ..., 10-5.5] = [-4.5, -3.5, ..., 4.5]
        # This is an array of size 10, starting at -4.5, with increment 1.0.
        expected_centered_x_array = ql.Array(10, -4.5, 1.0)

        delta_centered_x = x_for_autocorr - expected_centered_x_array
        dot_product_delta_centered_x = ql.DotProduct(delta_centered_x, delta_centered_x)

        if dot_product_delta_centered_x > 1.0e-6:
            self.fail(
                f"Centering error (x modified by autocorrelations):\n"
                f"  Calculated centered x: {list(x_for_autocorr)}\n"
                f"  Expected centered x:   {list(expected_centered_x_array)}\n"
                f"  DotProduct(delta, delta): {dot_product_delta_centered_x:.4e}"
            )
        self.assertArrayAlmostEqual(x_for_autocorr, expected_centered_x_array, delta=1e-7, msg="Centered x values differ.")


if __name__ == '__main__':
    print("Python QuantLib version:", ql.__version__)
    print("Testing Autocovariances (Python)...")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(AutocovariancesTests))
    unittest.TextTestRunner(verbosity=2).run(suite)